## Import necessary libraries

In [1]:
%reset -f

import pickle
import pandas as pd
import numpy
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score

## Save environment parameters, so those won't be saved to report file

In [2]:
environment_variables = dir()

## Define model related constants

In [3]:
# Input parameters
INPUT_TRAINING_FILE = "../data/preprocessed/difference_preprocessed_data_training.csv"
INPUT_EVALUATION_FILE = "../data/preprocessed/difference_preprocessed_data_evaluation.csv"

# Output parameters
METHOD_NAME = "svm"
TIMESTAMP = time.strftime("%d_%m_%Y-%H_%M_%S")
OUTPUT_MODEL = f"../data/models/{METHOD_NAME}_model_{TIMESTAMP}.pkl"
OUTPUT_RESULTS = f"../data/results/{METHOD_NAME}_model_{TIMESTAMP}.txt"

# Hyper parameter alternatives
HYPER_PARAMETER_KERNEL = ["rbf"]
HYPER_PARAMETER_GAMMA = ["scale"]
# Bankrupt companies must have higher weight, because there are fewer samples
# HYPER_PARAMETER_CLASS_WEIGHT = {0: 1.0, 1: 100.0}
HYPER_PARAMETER_CLASS_WEIGHT = "balanced"

# Hyper parameter optimization parameters
HYPER_PARAMETER_OPTIMIZATION_SCORING = "accuracy"
HYPER_PARAMETER_OPTIMIZATION_CV = 2

# Cross validation parameters
CROSS_VALIDATION_CV = 5

# Cost parameters
PUNISHMENT_FOR_FALSE_BANKRUPT_PREDICTION = 100

# Other constants
LABELS = ["Operational", "Bankrupt"]
RANDOM_SEED = 42

## Load the datasets
These dataset should contain "Bankrupt?" label and features that should be considered during learning.

In [4]:
training_dataset = pd.read_csv(INPUT_TRAINING_FILE, engine="python", delimiter=",")
training_features = training_dataset.loc[:, training_dataset.columns != "Bankrupt?"]
training_targets = training_dataset["Bankrupt?"]

evaluation_dataset = pd.read_csv(INPUT_EVALUATION_FILE, engine="python", delimiter=",")
evaluation_features = evaluation_dataset.loc[:, evaluation_dataset.columns != "Bankrupt?"]
evaluation_targets = evaluation_dataset["Bankrupt?"]

## Define the training pipeline

In [5]:
# Create internal pipeline
classifier = svm.SVC(class_weight=HYPER_PARAMETER_CLASS_WEIGHT, random_state=RANDOM_SEED)
pipeline = Pipeline(steps=[("classification", classifier)])

# Specify the tunable hyper parameters
parameters = {
    "classification__kernel": HYPER_PARAMETER_KERNEL,
    "classification__gamma": HYPER_PARAMETER_GAMMA
}

# Define KFold parameters
cv = StratifiedKFold(n_splits=HYPER_PARAMETER_OPTIMIZATION_CV, shuffle=True, random_state=42)

estimator = GridSearchCV(pipeline, parameters,
    scoring=HYPER_PARAMETER_OPTIMIZATION_SCORING, cv=cv)

## Do training

In [6]:
estimator.fit(training_features, training_targets)

GridSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('classification',
                                        SVC(class_weight='balanced',
                                            random_state=42))]),
             param_grid={'classification__gamma': ['scale'],
                         'classification__kernel': ['rbf']},
             scoring='accuracy')

## Calculate metric values

In [7]:
def confusion_matrix_to_string(cm):
    return f"True operational={cm[0][0]}, True bankrupt={cm[1][1]}, False operational={cm[0][1]}, False bankrupt={cm[1][0]}"

# Use all training data to calculate confusion matrix for training data
training_estimates = estimator.predict(training_features)
training_accuracy = balanced_accuracy_score(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix_to_string(training_confusion_matrix)

# Use model to estimate manually labeled evaluation Tweets
evaluation_estimates = estimator.predict(evaluation_features)
evaluation_accuracy = balanced_accuracy_score(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix_to_string(evaluation_confusion_matrix)

## Save trained model for future reference

In [8]:
with open(OUTPUT_MODEL, "wb") as handle:
    pickle.dump(estimator, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Save metadata and calculated statistics
This is done by saving every textual and numerical variable to file.

In [9]:
current_variables = dir()

results_string = ""

for variable in current_variables:
    # Skip environment variables and their container variable
    # Ignore also underscore variables
    if variable in environment_variables or variable == "environment_variables" or variable.startswith("_"):
        continue

    # Get variables value
    variable_value = globals()[variable]

    # If variable is numerical or string, append it to results
    if type(variable_value) is str or type(variable_value) is int or \
        type(variable_value) is float or type(variable_value) is list or \
        type(variable_value) is numpy.float64 or type(variable_value) is dict:
        results_string += f"{variable}: {variable_value}\n"

# Print results to screen
print(results_string)

# Save results to file
with open(OUTPUT_RESULTS, "w") as file:
    file.write(results_string)

CROSS_VALIDATION_CV: 5
HYPER_PARAMETER_CLASS_WEIGHT: balanced
HYPER_PARAMETER_GAMMA: ['scale']
HYPER_PARAMETER_KERNEL: ['rbf']
HYPER_PARAMETER_OPTIMIZATION_CV: 2
HYPER_PARAMETER_OPTIMIZATION_SCORING: accuracy
INPUT_EVALUATION_FILE: ../data/preprocessed/difference_preprocessed_data_evaluation.csv
INPUT_TRAINING_FILE: ../data/preprocessed/difference_preprocessed_data_training.csv
LABELS: ['Operational', 'Bankrupt']
METHOD_NAME: svm
OUTPUT_MODEL: ../data/models/svm_model_04_05_2022-17_00_47.pkl
OUTPUT_RESULTS: ../data/results/svm_model_04_05_2022-17_00_47.txt
PUNISHMENT_FOR_FALSE_BANKRUPT_PREDICTION: 100
RANDOM_SEED: 42
TIMESTAMP: 04_05_2022-17_00_47
evaluation_accuracy: 0.6977272727272728
evaluation_confusion_matrix: True operational=471, True bankrupt=15, False operational=189, False bankrupt=7
parameters: {'classification__kernel': ['rbf'], 'classification__gamma': ['scale']}
training_accuracy: 0.7853305746469579
training_confusion_matrix: True operational=4319, True bankrupt=167, Fals